6. Plot a time series line chart to show the trend of total installs over time,
segmented by app category. Highlight periods of significant growth by shading the
areas under the curve where the increase in installs exceeds 20% month-over-month
and content rating should be teen and app name should start with letter ‘E’ and 
installs should be more than 10k as well as this graph should work only between 6 PM
IST to 9 PM IST apart from that time we should not show this graph in dashboard itself.


In [19]:
from datetime import datetime
import pytz

# Define IST timezone and get current time in IST
ist = pytz.timezone('Asia/Kolkata')
current_time = datetime.now(ist)

# Check if current time is between 6 PM and 9 PM IST
start_time = current_time.replace(hour=18, minute=0, second=0, microsecond=0)
end_time = current_time.replace(hour=21, minute=0, second=0, microsecond=0)

if start_time <= current_time <= end_time:
    print("Proceeding to generate the graph...")

    # Load the data
    try:
        data = pd.read_csv('/mnt/data/Play Store Data.csv')

        # Clean and preprocess the data
        data['Installs'] = data['Installs'].str.replace('[+,]', '', regex=True).astype(float)
        data['Last Updated'] = pd.to_datetime(data['Last Updated'], errors='coerce')
        
        # Filter data based on the criteria
        filtered_data = data[
            (data['Content Rating'] == 'Teen') &
            (data['App'].str.startswith('E')) &
            (data['Installs'] > 10000)
        ]
        
        # Check if there's data after filtering
        if filtered_data.empty:
            print("No data meets the filtering criteria.")
        else:
            # Create time series data
            filtered_data['Year-Month'] = filtered_data['Last Updated'].dt.to_period('M')
            time_series = filtered_data.groupby(['Year-Month', 'Category'])['Installs'].sum().reset_index()
            time_series['Year-Month'] = time_series['Year-Month'].dt.to_timestamp()

            # Calculate month-over-month growth
            time_series['MoM Growth'] = time_series.groupby('Category')['Installs'].pct_change()

            # Plotting
            import matplotlib.pyplot as plt
            import seaborn as sns

            plt.figure(figsize=(14, 8))
            categories = time_series['Category'].unique()

            for category in categories:
                cat_data = time_series[time_series['Category'] == category]
                plt.plot(cat_data['Year-Month'], cat_data['Installs'], label=category)
                
                # Highlight growth > 20%
                growth_periods = cat_data[cat_data['MoM Growth'] > 0.2]
                plt.fill_between(
                    growth_periods['Year-Month'], 
                    0, 
                    growth_periods['Installs'], 
                    alpha=0.3, 
                    label=f"Significant Growth - {category}"
                )

            plt.title("Trend of Total Installs Over Time by Category")
            plt.xlabel("Time")
            plt.ylabel("Total Installs")
            plt.legend()
            plt.grid()
            plt.show()
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("The graph is only available between 6 PM and 9 PM IST.")


The graph is only available between 6 PM and 9 PM IST.
